In [2]:
### use the foolbox as the metric that we're minimizing for over hyperspace and see what kind of intersection we get between the two models

In [5]:
from hyperspace import create_hyperspace
from ray import tune
import tensorflow as tf
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers
from ray.tune.suggest.skopt import SkOptSearch
from skopt import Optimizer
import ray
from tqdm import tqdm
import torch
import torchvision
import statistics
import pandas as pd
import foolbox as fb
import numpy as np

### Tensorflow Tuning with Foolbox

In [9]:
def mnist_tf_objective(config):
    mnist = tf.keras.datasets.mnist

    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    model = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dropout(config['dropout']),
      tf.keras.layers.Dense(10, activation='softmax')
    ])

    opt = tf.keras.optimizers.Adam(learning_rate=config['learning_rate'])

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    res = model.fit(x_train, y_train, epochs=config['epochs'], batch_size=config['batch_size'])
    res_test = model.evaluate(x_test, y_test)
    fmodel = fb.TensorFlowModel(model, bounds=(0, 1))
    images, labels = fb.utils.samples(fmodel, dataset='mnist', batchsize=config['batch_size'])
    clean_accuracy = fb.utils.accuracy(fmodel, images, labels)
    attack = fb.attacks.GaussianBlurAttack()
    epsilons = [
        0.0,
        0.0002,
        0.0005,
        0.0008,
        0.001,
        0.0015,
        0.002,
        0.003,
        0.01,
        0.1,
        0.3,
        0.5,
        1.0,
    ]
    raw_advs, clipped_advs, success = attack(fmodel, images, labels, epsilons=epsilons)
    robust_accuracy = 1 - success.float32().mean(axis=-1)
    # res test[0] reports the loss from the evaluation, res_test[1] reports the accuracy
    tune.report(robust_acc = robust_accuracy)
    return robust_accuracy

In [10]:
### Defining the hyperspace
hyperparameters = [(0.00001, 0.1),  # learning_rate
                   (0.2, 0.9),  # dropout
                   (10, 100),  # epochs 
                   (10, 1000)]  # batch size
space = create_hyperspace(hyperparameters)

In [ ]:
%%capture tf_run_output

### for each space in hyperspace, we want to search the space using ray tune
results = []
for section in tqdm(space):
    # create a skopt gp minimize object
    optimizer = Optimizer(section)
    search_algo = SkOptSearch(optimizer, ['learning_rate', 'dropout', 'epochs', 'batch_size'],
                              metric='robust_acc', mode='max')
    # not using a gpu because running on local
    analysis = tune.run(mnist_tf_objective, search_alg=search_algo, num_samples=20, local_dir="~/Documents/resiliency/gaussianblur_tf_1")
    results.append(analysis)

 37/115 [========>.....................] - ETA: 0s - loss: 2.2792 - accuracy: 0.3844
